In [57]:
import cv2
import audioread
import logging
import os
import random
import time
import warnings

import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

from contextlib import contextmanager
from pathlib import Path
from typing import Optional

from fastprogress import progress_bar
from sklearn.metrics import f1_score
from torchvision import models

import cloudpickle

import matplotlib.pyplot as plt


In [58]:
!pip install ../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest/

Processing /kaggle/input/resnest50-fast-package/resnest-0.0.6b20200701/resnest
  Created wheel for resnest: filename=resnest-0.0.6b20200912-py3-none-any.whl size=31063 sha256=f3539d80f173054e32544fa34b0d8d5289c31c1fcc4d059035e9cb13f8f5018b
  Stored in directory: /tmp/pip-ephem-wheel-cache-74x1_sxv/wheels/23/50/5b/3bd292280af53184d3c3f652258ad6173224b2281eef8c4d27
Successfully built resnest
  Attempting uninstall: resnest
    Found existing installation: resnest 0.0.6b20200912
    Uninstalling resnest-0.0.6b20200912:
      Successfully uninstalled resnest-0.0.6b20200912


In [59]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
    
def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger
    
    
@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)

In [60]:
logger = get_logger("main.log")
set_seed(1213)

2020-09-12 03:06:52,355 - INFO - logger set up


In [61]:
TARGET_SR = 32000
TEST = Path("../input/birdsong-recognition/test_audio").exists()

In [62]:
if TEST:
    DATA_DIR = Path("../input/birdsong-recognition/")
else:
    # dataset created by @shonenkov, thanks!
    DATA_DIR = Path("../input/birdcall-custom/birdcall-check/")
    

test = pd.read_csv(DATA_DIR / "test.csv")
test_audio = DATA_DIR / "test_audio"


test.head()

,site,row_id,seconds,audio_id
0,site_1,site_1_41e6fe6504a34bf6846938ba78d13df1_5,5.0,41e6fe6504a34bf6846938ba78d13df1
1,site_1,site_1_41e6fe6504a34bf6846938ba78d13df1_10,10.0,41e6fe6504a34bf6846938ba78d13df1
2,site_1,site_1_41e6fe6504a34bf6846938ba78d13df1_15,15.0,41e6fe6504a34bf6846938ba78d13df1
3,site_1,site_1_41e6fe6504a34bf6846938ba78d13df1_20,20.0,41e6fe6504a34bf6846938ba78d13df1
4,site_1,site_1_41e6fe6504a34bf6846938ba78d13df1_25,25.0,41e6fe6504a34bf6846938ba78d13df1


In [63]:
sub = pd.read_csv("../input/birdsong-recognition/sample_submission.csv")
sub.to_csv("submission.csv", index=False)  # this will be overwritten if everything goes well

In [64]:
BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263
}

INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}

In [65]:
PERIOD = 5
MASK_NUM = 3
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

def mono_to_color(
    X: np.ndarray, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def normalize(image, mean, std):
    image = (image / 255.0).astype(np.float32)
    image = (image - mean) / std
    return np.moveaxis(image, 2, 0).astype(np.float32)

def plot_spec(melspec):
    plt.figure(figsize=(6,3), dpi= 80)
    melspec = librosa.power_to_db(melspec).astype(np.float32)
    plt.imshow(melspec)
    

def signal_to_mel(y, img_size):
    melspec = librosa.feature.melspectrogram(y,
                             sr=TARGET_SR,
                             fmin=20,
                             fmax=16000)
    melspec = librosa.power_to_db(melspec).astype(np.float32)
    image = mono_to_color(melspec)
    height, width, _ = image.shape
    image = cv2.resize(image, (int(width * img_size / height), img_size))
    image = normalize(image, mean=MEAN, std=STD)
    
    return image

def make_mask_images(melspec):

    len_mel = melspec.shape[1]
    hop_len = int(len_mel / (MASK_NUM*2))
    mels = []
    
    start = 0
    end = start + int(len_mel / MASK_NUM)
    mels.append(melspec)
    while end < len_mel:
        mask_mel = np.zeros((melspec.shape[0], melspec.shape[1]), dtype=melspec.dtype)
        mask_mel[:,start:end] = melspec[:,start:end]
        start += hop_len
        end = start + int(len_mel / MASK_NUM)
        mels.append(mask_mel)
#         plot_spec(mask_mel)
    
    return mels

def signal_to_mel_with_mask(y, img_size):
    melspec = librosa.feature.melspectrogram(y,
                             sr=TARGET_SR,
                             fmin=20,
                             fmax=16000)
    mask_mels = make_mask_images(melspec)
    images = []
    for melspec in mask_mels:
        melspec = librosa.power_to_db(melspec).astype(np.float32)
        image = mono_to_color(melspec)
        height, width, _ = image.shape
        image = cv2.resize(image, (int(width * img_size / height), img_size))
        image = normalize(image, mean=MEAN, std=STD)
        images.append(image)
    
    return images
    
class TestDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, clip: np.ndarray,
                 img_size):
#                  melspectrogram_parameters={}):
        self.df = df
        self.clip = clip
        self.img_size = img_size
#         self.melspectrogram_parameters = melspectrogram_parameters
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx: int):
        SR = TARGET_SR
        sample = self.df.loc[idx, :]
        site = sample.site
        row_id = sample.row_id
        
        if site == "site_3":
            y = self.clip.astype(np.float32)
#             _, y = librosa.effects.hpss(y, margin=(1.0,5.0))

            len_y = len(y)
            images = []
            effect_len = SR * PERIOD
            
            if(len_y < effect_len):
                for i in range(3):
                    new_y = np.zeros(effect_len, dtype=y.dtype)
                    start = np.random.randint(effect_len - len_y)
                    new_y[0:len(y)] = y
                    y = new_y.astype(np.float32)
                    image = signal_to_mel(y, self.img_size)
                    images.append(image)
            elif(len_y > effect_len):
                start = 0
                end = SR * PERIOD
                while len_y > start:
                    y_batch = y[start:end].astype(np.float32)
                    if len(y_batch) != (SR * PERIOD):
                        break
                    start = end
                    end = end + SR * PERIOD
                    image = signal_to_mel_with_mask(y_batch, self.img_size)
                    images.extend(image)
                    
                start = len(y) - SR * PERIOD
                end = len(y) 
                while 0 < start:
                    y_batch = y[start:end].astype(np.float32)
                    if len(y_batch) != (SR * PERIOD):
                        break
                    end = start
                    start = end - SR * PERIOD
                    image = signal_to_mel_with_mask(y_batch, self.img_size)
                    images.extend(image)
            else:
                y = y.astype(np.float32)
                image = signal_to_mel_with_mask(y, self.img_size)
                images.extend(image)
                
            images = np.asarray(images)

            return images, row_id, site
        else:
            end_seconds = int(sample.seconds)
            start_seconds = int(end_seconds - 5)
            
            start_index = SR * start_seconds
            end_index = SR * end_seconds
            
            y = self.clip[start_index:end_index].astype(np.float32)

            len_y = len(y)
            images = []      
            
            start = 0
            end = SR * PERIOD
            while len_y > start:
                y_batch = y[start:end].astype(np.float32)
                if len(y_batch) != (SR * PERIOD):
                    break
                start = end
                end = end + SR * PERIOD
                image = signal_to_mel_with_mask(y_batch, self.img_size)
                images.extend(image)
            
            ## melの秒数が5秒より小さいときは，逆側からもスライドさせる
            if(PERIOD < 5):
                start = len(y) - SR * PERIOD
                end = len(y) 
                while 0 < start:
                    y_batch = y[start:end].astype(np.float32)
                    if len(y_batch) != (SR * PERIOD):
                        break
                    end = start
                    start = end - SR * PERIOD
                    image = signal_to_mel_with_mask(y_batch, self.img_size)
                    images.extend(image)
                
            images = np.asarray(images)
            return images, row_id, site

def get_model():
    
    with open('../input/model-resnest-norm/model_ResNeSt_norm_best.pkl', 'rb') as f:
        model = cloudpickle.load(f)
#     model = ResNet(**config)
#     checkpoint = torch.load(weights_path)
#     model.load_state_dict(checkpoint["model_state_dict"])
    device = torch.device("cuda")
    model.to(device)
    model.eval()
    return model

In [66]:
def get_model():
    
    with open('../input/model-resnest-norm/model_ResNeSt_norm_best.pkl', 'rb') as f:
        model = cloudpickle.load(f)
#     model = ResNet(**config)
#     checkpoint = torch.load(weights_path)
#     model.load_state_dict(checkpoint["model_state_dict"])
    device = torch.device("cuda")
    model.to(device)
    model.eval()
    return model

In [98]:
def prediction_for_clip(test_df: pd.DataFrame, 
                        clip: np.ndarray, 
                        model, 
#                         mel_params: dict, 
                        threshold):
    dataset = TestDataset(df=test_df, 
                          clip=clip,
                          img_size=224)
#                           melspectrogram_parameters=mel_params)
    loader = data.DataLoader(dataset, batch_size=1, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.eval()
    prediction_dict = {}
    for image, row_id, site in progress_bar(loader):
        site = site[0]
        row_id = row_id[0]
#         if site in {"site_1", "site_2"}:
#             image = image.to(device)

#             with torch.no_grad():
#                 prediction = model(image)
#                 prediction = F.sigmoid(prediction)
                
#                 proba = prediction.detach().cpu().numpy()

#             events = proba >= threshold
#             labels = np.argwhere(events[0]).reshape(-1).tolist()

#         else:
        image = image.squeeze(0)
        batch_size = 16
        whole_size = image.size(0)
        if whole_size % batch_size == 0:
            n_iter = whole_size // batch_size
        else:
            n_iter = whole_size // batch_size + 1

        all_events = set()
        nocall_flag = False
        
        if site in {"site_1", "site_2"}:
            #一枚目(全体)チェック
            batch = image[:1]
            if batch.to('cpu').detach().numpy().copy().ndim == 3:
                batch = batch.unsqueeze(0)
            batch = batch.to(device)
            with torch.no_grad():
                prediction = model(batch)
                prediction = F.sigmoid(prediction)
                proba = prediction.detach().cpu().numpy()
            nocall_proba = proba[0]
            nocall_event = nocall_proba >= threshold
            nocall_label = np.argwhere(nocall_event).reshape(-1).tolist()
            if len(nocall_label) == 0:
                nocall_flag = True
    #         print(nocall_label)

        if nocall_flag == False:

            for batch_i in range(n_iter):
                batch = image[batch_i * batch_size:(batch_i + 1) * batch_size]
    #             print(batch.shape)
                if batch.to('cpu').detach().numpy().copy().ndim == 3:
                    batch = batch.unsqueeze(0)
                batch = batch.to(device)
                with torch.no_grad():
                    prediction = model(batch)
                    prediction = F.sigmoid(prediction)
                    proba = prediction.detach().cpu().numpy()
                    events = proba >= threshold
                    for i in range(len(events)):
                        event = events[i, :]
                        labels = np.argwhere(event).reshape(-1).tolist()
                        for label in labels:
                            all_events.add(label)

        labels = list(all_events)
        if len(labels) == 0:
            prediction_dict[row_id] = "nocall"
        else:
            labels_str_list = list(map(lambda x: INV_BIRD_CODE[x], labels))
            label_string = " ".join(labels_str_list)
            prediction_dict[row_id] = label_string
    return prediction_dict

In [99]:
def prediction(test_df: pd.DataFrame,
               test_audio: Path,
               threshold):
#     model = get_model(model_config)
    model = get_model()
    unique_audio_id = test_df.audio_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    for audio_id in unique_audio_id:
        with timer(f"Loading {audio_id}", logger):
            clip, _ = librosa.load(test_audio / (audio_id + ".mp3"),
                                   sr=TARGET_SR,
                                   mono=True,
                                   res_type="kaiser_fast")
#             _, clip = librosa.effects.hpss(clip, margin=(1.0,5.0))
        
        test_df_for_audio_id = test_df.query(
            f"audio_id == '{audio_id}'").reset_index(drop=True)
        with timer(f"Prediction on {audio_id}", logger):
            prediction_dict = prediction_for_clip(test_df_for_audio_id,
                                                  clip=clip,
                                                  model=model,
#                                                   mel_params=mel_params,
                                                  threshold=threshold)
        row_id = list(prediction_dict.keys())
        birds = list(prediction_dict.values())
        prediction_df = pd.DataFrame({
            "row_id": row_id,
            "birds": birds
        })
        prediction_dfs.append(prediction_df)
        
#         break
    
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df

In [100]:
submission = prediction(test_df=test,
                        test_audio=test_audio,
#                         mel_params=melspectrogram_parameters,
#                         weights_path=weights_path,
                        threshold=0.6)
submission.to_csv("submission.csv", index=False)

2020-09-12 03:24:19,952 - INFO - [Loading 41e6fe6504a34bf6846938ba78d13df1] start
2020-09-12 03:24:20,635 - INFO - [Loading 41e6fe6504a34bf6846938ba78d13df1] done in 0.68 s
2020-09-12 03:24:20,643 - INFO - [Prediction on 41e6fe6504a34bf6846938ba78d13df1] start


2020-09-12 03:24:21,176 - INFO - [Prediction on 41e6fe6504a34bf6846938ba78d13df1] done in 0.53 s
2020-09-12 03:24:21,179 - INFO - [Loading cce64fffafed40f2b2f3d3413ec1c4c2] start
2020-09-12 03:24:21,948 - INFO - [Loading cce64fffafed40f2b2f3d3413ec1c4c2] done in 0.77 s
2020-09-12 03:24:21,955 - INFO - [Prediction on cce64fffafed40f2b2f3d3413ec1c4c2] start


2020-09-12 03:24:22,550 - INFO - [Prediction on cce64fffafed40f2b2f3d3413ec1c4c2] done in 0.59 s
2020-09-12 03:24:22,553 - INFO - [Loading 99af324c881246949408c0b1ae54271f] start
2020-09-12 03:24:23,385 - INFO - [Loading 99af324c881246949408c0b1ae54271f] done in 0.83 s
2020-09-12 03:24:23,392 - INFO - [Prediction on 99af324c881246949408c0b1ae54271f] start


2020-09-12 03:24:24,127 - INFO - [Prediction on 99af324c881246949408c0b1ae54271f] done in 0.73 s
2020-09-12 03:24:24,129 - INFO - [Loading 6ab74e177aa149468a39ca10beed6222] start
2020-09-12 03:24:24,849 - INFO - [Loading 6ab74e177aa149468a39ca10beed6222] done in 0.72 s
2020-09-12 03:24:24,856 - INFO - [Prediction on 6ab74e177aa149468a39ca10beed6222] start


2020-09-12 03:24:25,430 - INFO - [Prediction on 6ab74e177aa149468a39ca10beed6222] done in 0.57 s
2020-09-12 03:24:25,433 - INFO - [Loading b2fd3f01e9284293a1e33f9c811a2ed6] start
2020-09-12 03:24:26,483 - INFO - [Loading b2fd3f01e9284293a1e33f9c811a2ed6] done in 1.05 s
2020-09-12 03:24:26,489 - INFO - [Prediction on b2fd3f01e9284293a1e33f9c811a2ed6] start


...

2020-09-12 03:24:48,598 - INFO - [Prediction on 9cc5d9646f344f1bbb52640a988fe902] done in 9.63 s
2020-09-12 03:24:48,603 - INFO - [Loading a56e20a518684688a9952add8a9d5213] start
2020-09-12 03:24:49,665 - INFO - [Loading a56e20a518684688a9952add8a9d5213] done in 1.06 s
2020-09-12 03:24:49,673 - INFO - [Prediction on a56e20a518684688a9952add8a9d5213] start


2020-09-12 03:24:50,689 - INFO - [Prediction on a56e20a518684688a9952add8a9d5213] done in 1.02 s
2020-09-12 03:24:50,691 - INFO - [Loading 96779836288745728306903d54e264dd] start
2020-09-12 03:24:51,275 - INFO - [Loading 96779836288745728306903d54e264dd] done in 0.58 s
2020-09-12 03:24:51,282 - INFO - [Prediction on 96779836288745728306903d54e264dd] start


2020-09-12 03:24:51,797 - INFO - [Prediction on 96779836288745728306903d54e264dd] done in 0.52 s
2020-09-12 03:24:51,799 - INFO - [Loading f77783ba4c6641bc918b034a18c23e53] start
2020-09-12 03:24:51,832 - INFO - [Loading f77783ba4c6641bc918b034a18c23e53] done in 0.03 s
2020-09-12 03:24:51,836 - INFO - [Prediction on f77783ba4c6641bc918b034a18c23e53] start


2020-09-12 03:24:51,927 - INFO - [Prediction on f77783ba4c6641bc918b034a18c23e53] done in 0.09 s
2020-09-12 03:24:51,929 - INFO - [Loading 856b194b097441958697c2bcd1f63982] start
2020-09-12 03:24:52,610 - INFO - [Loading 856b194b097441958697c2bcd1f63982] done in 0.68 s
2020-09-12 03:24:52,617 - INFO - [Prediction on 856b194b097441958697c2bcd1f63982] start


2020-09-12 03:24:53,513 - INFO - [Prediction on 856b194b097441958697c2bcd1f63982] done in 0.90 s


In [101]:
pd.set_option('display.max_rows', 500)
submission.tail(76)

,row_id,birds
0,site_1_41e6fe6504a34bf6846938ba78d13df1_5,aldfly lobdow
1,site_1_41e6fe6504a34bf6846938ba78d13df1_10,nocall
2,site_1_41e6fe6504a34bf6846938ba78d13df1_15,aldfly amered
3,site_1_41e6fe6504a34bf6846938ba78d13df1_20,bkbwar amered
4,site_1_41e6fe6504a34bf6846938ba78d13df1_25,aldfly
5,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_5,aldfly
6,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_10,nocall
7,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_15,nocall
8,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_20,nocall
9,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_25,aldfly lobdow
